In [1]:
import warnings
import random
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn import linear_model
%matplotlib inline

In [2]:
%%capture
!wget https://www.dropbox.com/s/afwb0tnqm9izxha/predict_house_price_training_data.xlsx
!wget https://www.dropbox.com/s/sur2avqf4n5f4az/predict_house_price_test_data.xlsx

In [3]:
training_data = pd.read_excel('predict_house_price_training_data.xlsx') # загружаем таблицу в переменную training_data

In [4]:
training_data.head()

,Целевая.Цена,Спальни,Ванные,Жилая площадь,Общая площадь,Количество этажей,Вид на воду,Просмотрены ранее,Состояние,Оценка риелтора,Площадь без подвала,Площадь подвала,Год постройки,Год реновации,Широта,Долгота
0,830000,5,3.50,3490,21780,2.0,0,0,3,8,3490,0,1996,0,47.6707,-122.144
1,385000,4,1.75,2360,7620,1.0,0,0,4,7,1180,1180,1955,0,47.5278,-122.345
2,610000,6,2.75,2040,8560,1.0,0,2,4,7,1100,940,1961,0,47.6160,-122.115
3,550000,3,1.75,1940,8376,1.0,0,0,4,8,1290,650,1963,0,47.5586,-122.173
4,1300000,3,2.75,3450,5350,1.5,0,3,4,9,2590,860,1925,0,47.6389,-122.407


In [5]:
target_variable_name = 'Целевая.Цена'
training_values = training_data[target_variable_name]
training_points = training_data.drop(target_variable_name, axis=1)
training_points

,Спальни,Ванные,Жилая площадь,Общая площадь,Количество этажей,Вид на воду,Просмотрены ранее,Состояние,Оценка риелтора,Площадь без подвала,Площадь подвала,Год постройки,Год реновации,Широта,Долгота
0,5,3.50,3490,21780,2.0,0,0,3,8,3490,0,1996,0,47.6707,-122.144
1,4,1.75,2360,7620,1.0,0,0,4,7,1180,1180,1955,0,47.5278,-122.345
2,6,2.75,2040,8560,1.0,0,2,4,7,1100,940,1961,0,47.6160,-122.115
3,3,1.75,1940,8376,1.0,0,0,4,8,1290,650,1963,0,47.5586,-122.173
4,3,2.75,3450,5350,1.5,0,3,4,9,2590,860,1925,0,47.6389,-122.407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,4,4.25,5710,24663,2.0,0,0,3,11,5710,0,2007,0,47.3065,-122.158
15125,3,1.50,1950,4473,1.0,0,0,4,6,1530,420,1914,0,47.7094,-122.228
15126,3,2.00,3620,14250,2.0,0,0,3,8,3220,400,2007,0,47.5767,-122.225
15127,4,2.50,2340,9748,1.0,0,1,3,8,1610,730,1981,0,47.3363,-122.331


In [6]:
training_points.shape

(15129, 15)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

training_points = scaler.fit_transform(training_points)
sthgd_model= linear_model.SGDRegressor()
sthgd_model.fit(training_points, training_values)
w_true = sthgd_model.coef_
b = sthgd_model.intercept_
print(sthgd_model.score(training_points,training_values))

0.6904969552774036


In [8]:
def q_i(i,w, training_points, training_values, a): # функция для вычисления градиента каждого объекта под номером i
  q = []
  f = w[0]
  for h in range(1,15):
    f += w[h]*training_points[i,h-1]
  f = f - training_values[i]
  q.append(f*2+2*a*w[0])
  for j in range(1,15):
    q.append(2*f*training_points[i,j-1]+a*w[j])
  return q

In [9]:
def q_n(q): #функция нахождения абсолютной величины градиента
  q_nn=0
  for i in q:
    q_nn = q_nn + i**2
  q_nn = q_nn**(0.5)
  return q_nn

In [10]:
k=1
w = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
w_k = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
a=0.01
i = random.randint(0, 15128)
q = q_i(i,w_k, training_points, training_values, a)
for j in range(len(w)):
  w_k[j] = w[j]*(1-(1/k)*a) - (1/k)*q[j]

while q_n(w_k) - q_n(w) > 0.00001: #реализуем алгоритм стохастического градиентного спуска
  w = w_k.copy()
  i = random.randint(0, 15128)
  q = q_i(i,w_k, training_points, training_values,a)
  for j in range(len(w)):
    w_k[j] = w[j]*(1-(1/k)*a) - (1/k)*q[j] # попробовать еще раз с регуляризацией
  k+=1
w_k = np.asarray(w_k)
print(w_k)

[  6654101.90051703   5742105.34776453  17425511.8783679
   3331599.50387646  -2016881.84707865  19289493.10415786
   -576640.15903141  -2045341.82413758  -4205508.3709141
  -3705619.12491131   7521284.35020624  -7229976.12453079
  14034616.66347824   -509650.86155294 -16221185.33744616]


In [11]:
y_est = []
for i in range(0,15129):
  y = sum(w_k * training_points[i])
  y_est.append(y)
y_est = np.asarray(y_est)

In [12]:
b = np.average(training_values - y_est)
b

538795.2958567592

In [15]:
def my_reg_model_predict(training_points):
  training_points = np.asarray(training_points)
  predicted_values = []
  for i in range(len(training_points)):
    y = sum(w_k * training_points[i]) + b
    predicted_values.append(y)
  predicted_values = np.asarray(predicted_values)
  return predicted_values
new_a = []
for i in range(0,15129):
  new_a.append(120000)

from sklearn.metrics import mean_squared_error, mean_absolute_error
print(type(training_points))
print('My Train MSE: ', mean_squared_error(training_values, my_reg_model_predict(training_points)))
print('My Train MSE: ', mean_squared_error(training_values, new_a))
sk_predicted = sthgd_model.predict(training_points)
print('Sk Train MSE: ', mean_squared_error(training_values, sk_predicted))

<class 'numpy.ndarray'>
My Train MSE:  1688249666922232.2
My Train MSE:  309774654760.14484
Sk Train MSE:  41592614616.14563


In [ ]:
u = ((training_values - predicted_values)** 2).sum()
v = ((training_values - predicted_values.mean())** 2).sum()
print('R value for my model is ', 1-u/v)
print('R value for sk model is', sthgd_model.score(training_points,training_values))

Теперь протестируем модель линейной регрессии от sklearn

In [ ]:
linear_regression_model = linear_model.LinearRegression()
linear_regression_model.fit(training_points, training_values)

In [ ]:
test_data = pd.read_excel('predict_house_price_test_data.xlsx')
test_data.head()

In [ ]:
test_data.shape

In [ ]:
test_values = test_data[target_variable_name]
test_points = test_data.drop(target_variable_name, axis=1)
test_points.head()

In [ ]:
test_predictions_linear = linear_regression_model.predict(test_points)
my_test_predictions = my_reg_model_predict(test_points)
print('My test MSE: ', mean_squared_error(test_values, my_test_predictions))
print('Sk reg test MSE', mean_squared_error(test_values, test_predictions_linear))
print('My test MAE: ', mean_absolute_error(test_values, my_test_predictions))
print('Sk reg test MAE', mean_absolute_error(test_values, test_predictions_linear))

In [ ]:
test_points_n = scaler.transform(test_points)
test_predictions_linear = linear_regression_model.predict(test_points_n)
plt.figure(figsize=(25, 8))
plt.bar(test_points.columns, linear_regression_model.coef_)

In [ ]:
d = {x: y for x, y in zip(list(test_points.columns), [abs(x) for x in list(linear_regression_model.coef_)])}
feature_importance = pd.DataFrame.from_dict(d, orient='index', columns = ['Важность признака'])
print(feature_importance)

In [ ]:
feature_importance.sort_values(by='Важность признака', ascending=False)

In [ ]:
train_predictions_linear = linear_regression_model.predict(training_points)
anomalies_in_train = []
for i in range(len(training_values)):
  if train_predictions_linear[i] - training_values[i] > mean_absolute_error(training_values, train_predictions_linear):
    anomalies_in_train.append(i)
print("Indexes of anomalies in training data: ", anomalies_in_train)
anomalies_in_test = []
for i in range(len(test_values)):
  if test_predictions_linear[i] - test_values[i] > mean_absolute_error(test_values, test_predictions_linear):
    anomalies_in_test.append(i)
print("Indexes of anomalies in test data: ", anomalies_in_test)